# Scénario

Vous êtes consultant chez LaPage, une grande librairie généraliste en ligne très réputée. Vous êtes directement rattaché au service Marketing. Lapage était originellement une librairie physique avec points de vente. Mais devant le succès de certains de ses produits et l'engouement de ses clients, elle a décidé depuis 2 ans d'ouvrir un site de vente en ligne. Vous intervenez car l'entreprise souhaite faire le point après deux ans d'exercice, et pouvoir analyser ses points forts, ses points faibles, les comportements cients et etc. Nous aimerions faire le point à présent. Cela nous permettra de décider de la marche à suivre, par exemple décider si nous devons créer certaines offres, adapter certains prix, etc.

Cette analyse va être découpée en deux parties :
<ol>
    <li>Une analyse des différents indicateurs de vente, qui a été demandée directement.</li>
    <ol>
        <li>Dans un premier temps, on va avoir besoin de différents indicateurs et graphiques autour du chiffre d'affaires. Peux-tu également regarder l'évolution dans le temps et mettre en place une décomposition en moyenne mobile pour évaluer la tendance globale ? Il serait également intéressant de faire un zoom sur les références, pour voir un peu les tops et les flops, la répartition par catégorie, etc.</li>
        <li>Enfin, on aimerait avoir quelques informations sur les profils de nos clients, et également la répartition du chiffre d'affaires entre eux, via par exemple une courbe de Lorenz.</li>
    </ol>
    <li>Une analyse plus ciblée sur les clients : l'objectif serait cette fois-ci de comprendre le comportement de nos clients en ligne, pour pouvoir ensuite comparer avec la connaissance acquise via nos librairies physiques. On aimera que tu regardes en particulier :</li>
    <ol>
        <li>le lien entre le genre d'un client et les catégories des livres achetés;</li>
        <li>et ensuite le lient entre l'âge des clients et le montant total des achats, la fréquance d'achat, la taille du panier moyen et les catégories des livres achetés</li>
    </ol>
</ol>

# Remarques

<ol>
    <li>Détail du nettoyage des données : vous présenterez l'ensemble de données exploitable.</li>
    <li>Restitution de l'analyse : les chiffres clés, KPI, graphiques générés corrélation calculées, etc. (le calcul de l'indicateur de corrélation (R²; eta², Chi²) et la mise en place d'un test statistique est nécessaire)

# Ressources

<ol>
    <li>Réalisez des graphiques compréhensibles <font color = aqua>https://openclassrooms.com/fr/courses/4525336-realisez-des-rapports-statistiques-clairs-et-impactants/5193755-choisissez-des-graphiques-adaptes-a-votre-message</font></li>
    <li>Maitrisez les bases des probabilites <font color = cyan>https://openclassrooms.com/fr/courses/4525296-maitrisez-les-bases-des-probabilites</font></li>
    <li>Realisez des rapports statistiques clairs et impactants <font color = cyan>https://openclassrooms.com/fr/courses/4525336-realisez-des-rapports-statistiques-clairs-et-impactants</font></li>
    <li>Initiez vous a la statistique inferentielle <font color = cyan>https://openclassrooms.com/fr/courses/4525306-initiez-vous-a-la-statistique-inferentielle</font></li>

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
df_customers = pd.read_csv('data/customers.csv')
df_products = pd.read_csv('data/products.csv')
df_transactions = pd.read_csv('data/transactions.csv')

In [24]:
df = pd.merge(df_customers, df_transactions, how='left', on='client_id')
df = pd.merge(df, df_products, how='left', on='id_prod')
df.shape

(679553, 8)

In [28]:
df.columns

Index(['client_id', 'sex', 'birth', 'id_prod', 'date', 'session_id', 'price',
       'categ'],
      dtype='object')

In [29]:
chiffre_affaire = df['price'].sum() 

print(f"Le chiffre d'affaire est de {chiffre_affaire} €")

Le chiffre d'affaire est de 11853528.68 €
